In [162]:
!rm -rf my_package
!mkdir my_package

In [163]:
%%writefile my_package/__init__.py

Writing my_package/__init__.py


In [164]:
%%writefile my_package/final_project.py

import math
import time
import random
import sobol_seq
import multiprocessing as mp

random.seed(410)


# Section 2

## Section 2.1
def leibniz_pi(iterations):
    pi_est = 0.0
    for n in range(iterations):
        pi_est += ((-1)**n) / (2*n + 1)
    return 4 * pi_est

## Section 2.2
def integrand(x):
    """Compute sqrt(1 - x^2), the integrand corresponding to the quarter circle."""
    return (1 - x**2) ** 0.5

def trapezoidal_integration(a, b, N):
    """
    Approximate the integral of f(x) from x=a to x=b using the trapezoidal rule with N subintervals.
    Returns 4 * (approximate integral) to estimate π.
    """
    h = (b - a) / N
    s = 0.5 * (integrand(a) + integrand(b))
    for k in range(1, N):
        s += integrand(a + k * h)
    return 4 * s * h  # Multiply by 4 to obtain π

## Section 2.3
def monte_carlo_pi(samples):
    """Estimate π using Monte Carlo by sampling 'samples' points in [0,1]x[0,1]."""
    inside_circle = 0
    for _ in range(samples):
        x = random.random()
        y = random.random()
        if x**2 + y**2 <= 1:
            inside_circle += 1
    return 4 * (inside_circle / samples)

## Section 2.4
def mcmc_pi(samples, step_size=0.1):
    """
    Estimate π using a simple random-walk MCMC approach within the unit square.

    Parameters:
    samples   (int) : Number of MCMC steps to perform
    step_size (float): Max step size in both x and y directions for each proposal

    Returns:
    float: The estimated value of π after 'samples' MCMC steps.
    """
    # Initialize the chain at the center of the unit square (0.5, 0.5)
    x, y = 0.5, 0.5
    inside_circle = 0

    for _ in range(samples):
        # Propose new point
        x_new = x + random.uniform(-step_size, step_size)
        y_new = y + random.uniform(-step_size, step_size)

        # Check if new point is within bounds
        if 0 <= x_new <= 1 and 0 <= y_new <= 1:
            # Accept the new point
            x, y = x_new, y_new
        # Count if point is inside quarter-circle
        if x**2 + y**2 <= 1:
            inside_circle += 1

    return 4 * (inside_circle / samples)


# Section 3
## Section 3.1.1
def leibniz_pi_kahan(iterations):
    """
    Compute the Leibniz series approximation of π using Kahan summation
    to reduce floating-point round-off error.

    Parameters:
    iterations (int): Number of terms to sum in the Leibniz series

    Returns:
    float: The approximation of π after 'iterations' terms
    """
    pi_est = 0.0
    c = 0.0  # compensation
    for n in range(iterations):
        # term = ((-1)**n) / (2*n + 1)
        term = (1.0 if (n % 2 == 0) else -1.0) / (2*n + 1)

        # Kahan Summation
        y = term - c
        t = pi_est + y
        c = (t - pi_est) - y
        pi_est = t

    return 4 * pi_est

def leibniz_partial_sum(N):
    """Compute the partial sum S_N of the Leibniz series for π/4."""
    s = 0.0
    for n in range(N):
        s += ((-1)**n)/(2*n+1)
    return s

## Section 3.1.2
def aitken(N):
    """Apply one step of Aitken's Δ² acceleration to the Leibniz series partial sums."""
    S0 = leibniz_partial_sum(N)
    S1 = leibniz_partial_sum(N+1)
    S2 = leibniz_partial_sum(N+2)

    d0 = S1 - S0  # ΔS0
    d1 = S2 - S1  # ΔS1
    d2 = d1 - d0  # Δ^2S0

    # Aitken’s Formula
    # S0* = S0 - (d0^2 / d2)
    if abs(d2) < 1e-18:
        # Avoid division by zero in degenerate cases
        S0_star = S2
    else:
        S0_star = S0 - (d0 * d0 / d2)

    return 4.0 * S0_star  # multiply by 4 for pi

## Section 3.2.1
def simpsons_rule_integration(a, b, N):
    # N must be even
    if N % 2 != 0:
        N += 1

    h = (b - a) / N
    s = integrand(a) + integrand(b)

    # Accumulate terms with alternating factors of 4 and 2
    for k in range(1, N):
        xk = a + k*h
        if k % 2 == 1:
            s += 4 * integrand(xk)
        else:
            s += 2 * integrand(xk)

    return (h/3.0) * s

## Section 3.2.2
def monte_carlo_integration(num_samples):
    count_inside = 0

    for _ in range(num_samples):
        x = random.random()
        y = random.random()

        # Check if point is under quarter circle
        if x*x + y*y <= 1.0:
            count_inside += 1

    return 4.0 * count_inside / num_samples

## Section 3.3.1
def stratified_sampling_pi(strata, samples_per_stratum):
    inside_circle = 0
    total_samples = strata * strata * samples_per_stratum

    # size of each sub-square
    step = 1.0 / strata

    for i in range(strata):
        for j in range(strata):
            for _ in range(samples_per_stratum):
                # pick random point inside [i*step, (i+1)*step] x [j*step, (j+1)*step]
                x = (i + random.random()) * step
                y = (j + random.random()) * step

                if x*x + y*y <= 1.0:
                    inside_circle += 1

    return 4.0 * inside_circle / total_samples

## Section 3.3.2
def quasi_monte_carlo_pi(num_samples):
    inside_circle = 0
    # Generate sobol points in 2 dimensions
    points = sobol_seq.i4_sobol_generate(2, num_samples)
    for (x, y) in points:
        if x*x + y*y <= 1.0:
            inside_circle += 1
    return 4.0 * inside_circle / num_samples

## Section 3.3.3
def sample_mc_3d(num_points):
    """
    Estimate the value of π using a Monte Carlo method by sampling points
    inside a cube [-1, 1]^3 and checking how many lie within the unit sphere.

    :param num_points: Number of random points to sample.
    :return: Approximate value of π.
    """
    count_inside = 0
    for _ in range(num_points):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        z = random.uniform(-1, 1)
        if x*x + y*y + z*z <= 1:
            count_inside += 1

    fraction = count_inside / num_points
    pi_estimate = 6 * fraction
    return pi_estimate

## Section 3.3.4
def sample_gaussian_2d(sigma=0.5):
    """
    Draw a single sample (X, Y) from a 2D Gaussian with mean (0,0) and
    diagonal covariance matrix sigma^2 * I.

    Using the Box-Muller transform for two standard normals, then scale by sigma.
    """
    u1 = random.random()
    u2 = random.random()
    # Standard normal using Box-Muller
    r = math.sqrt(-2.0 * math.log(u1))
    theta = 2.0 * math.pi * u2
    z1 = r * math.cos(theta)
    z2 = r * math.sin(theta)
    # Scale by sigma
    return sigma * z1, sigma * z2

def gauss_pdf(x, y, sigma=0.5):
    """
    The value of the 2D Gaussian PDF with mean (0,0) and variance sigma^2 (isotropic).
    q(x,y) = (1 / (2*pi*sigma^2)) * exp(-(x^2 + y^2)/(2*sigma^2))
    """
    coeff = 1.0 / (2.0 * math.pi * sigma * sigma)
    exponent = - (x*x + y*y) / (2.0 * sigma*sigma)
    return coeff * math.exp(exponent)

def estimate_pi_importance_gaussian(N, sigma=0.5):
    """
    Estimate pi using Importance Sampling with a 2D Gaussian proposal distribution.

    Steps:
      1. Generate N samples (x,y) ~ Gaussian(0, sigma^2 I)
      2. For each sample, compute the indicator if it's inside the unit circle
         and the importance weight w = 1/q(x,y)
      3. Sum up weights for those inside the circle
      4. pi ~ 4 * (average of those weights)
    """
    w_sum = 0.0
    for _ in range(N):
        x, y = sample_gaussian_2d(sigma)
        # Check if inside unit circle
        inside = (x*x + y*y <= 1.0)
        if inside:
            # Importance weight = 1 / q(x,y)
            q_val = gauss_pdf(x, y, sigma)
            # If q_val is extremely small, watch out for floating-point overflow in 1/q_val
            w_sum += 1.0 / q_val
        # If not inside, weight = 0 (i.e. does not contribute)

    pi_est = (w_sum / N)
    return pi_est

## Section 3.4.1
def mcmc_pi_adaptive(samples, initial_step=0.1, target_accept=0.3, adapt_interval=1000):
    """
    MCMC approach to estimate π in the unit square [0,1] x [0,1].
    Adaptively adjust the step size to maintain a target acceptance rate.

    :param samples: total number of MCMC steps
    :param initial_step: initial step size for x, y moves
    :param target_accept: target acceptance ratio (e.g. 0.3 ~ 0.5)
    :param adapt_interval: how often (in steps) we recalculate acceptance and adapt step size
    :return: approximate value of π
    """
    x, y = 0.5, 0.5  # start near the center
    step_size = initial_step

    inside_circle = 0
    accepted = 0  # count how many proposals are accepted

    start_time = time.time()

    for i in range(1, samples + 1):
        # Propose new point
        x_new = x + random.uniform(-step_size, step_size)
        y_new = y + random.uniform(-step_size, step_size)

        # Metropolis acceptance if in [0,1]^2
        if 0 <= x_new <= 1 and 0 <= y_new <= 1:
            x, y = x_new, y_new
            accepted += 1

        # Check if inside quarter circle
        if x*x + y*y <= 1.0:
            inside_circle += 1

        # Every adapt_interval steps, adjust step_size
        if i % adapt_interval == 0:
            acc_rate = accepted / adapt_interval
            # If acceptance > target, increase step; else decrease step
            if acc_rate > target_accept:
                step_size *= 1.1  # enlarge step
            else:
                step_size *= 0.9  # shrink step
            accepted = 0  # reset accepted count

    pi_estimate = 4.0 * inside_circle / samples
    return pi_estimate

## Section 3.4.2
def mcmc_chain(samples, step_size=0.1, seed=None):
    """
    Single-chain MCMC for pi estimation. Returns the number of points inside circle.
    We separate 'inside' count from total so we can combine results later.
    """
    if seed is not None:
        random.seed(seed)

    x, y = 0.5, 0.5
    inside_circle = 0

    for _ in range(samples):
        x_new = x + random.uniform(-step_size, step_size)
        y_new = y + random.uniform(-step_size, step_size)

        if 0 <= x_new <= 1 and 0 <= y_new <= 1:
            x, y = x_new, y_new

        if x*x + y*y <= 1.0:
            inside_circle += 1

    return inside_circle

Writing my_package/final_project.py


In [165]:
%%writefile setup.py
from setuptools import setup, find_packages

setup(
    name='my_package',
    version='0.1.0',
    packages=find_packages(),
    install_requires=[
        'sobol_seq',  # 外部ライブラリだけ
    ],
    author='Tatsuya Shiokawa',
    author_email='tatsuya.shiokawa@mail.utoronto.ca',
    description='Final Project',
    url='https://github.com/ShioTatsu-Japan/STA410H1',
    classifiers=[
        'Programming Language :: Python :: 3',
        'License :: OSI Approved :: MIT License',
    ],
    python_requires='>=3.6',
)

Overwriting setup.py


In [166]:
!pip install -e .

Obtaining file:///content
  Preparing metadata (setup.py) ... done
  Attempting uninstall: my_package
    Found existing installation: my_package 0.1.0
    Uninstalling my_package-0.1.0:
      Successfully uninstalled my_package-0.1.0
  Running setup.py develop for my_package


In [167]:
from my_package.final_project import leibniz_pi, integrand, trapezoidal_integration, \
monte_carlo_pi, mcmc_pi, leibniz_pi_kahan, leibniz_partial_sum, aitken, \
simpsons_rule_integration, monte_carlo_integration, stratified_sampling_pi, \
quasi_monte_carlo_pi, sample_mc_3d, sample_gaussian_2d, gauss_pdf, \
estimate_pi_importance_gaussian, mcmc_pi_adaptive, mcmc_chain
import math
import time
import random
import sobol_seq
import multiprocessing as mp

# 0. Table of Contents

### 1. Introduction and Motivation

### 2. Four Methods for Estimating $\pi$
  * #### 2.1. Leibniz Series
  * #### 2.2. Integral-Based Estimation
  * #### 2.3. Classic Monte Carlo
  * #### 2.4. Markov Chain Monte Carlo (MCMC)

### 3. Advanced Techniques and Performance Analysis
  * #### 3.1. Advanced Techniques for Leibniz Series
    * 3.1.1. Kahan Summation
    * 3.1.2. Aitken Formula
  * #### 3.2. Advanced Techniques for Integral-Based Estimation
    * 3.2.1. Simpson's Rule
    * 3.2.2. Monte Carlo Integration
  * #### 3.3. Advanced Techniques for Classic Monte Carlo
    * 3.3.1. Stratified Sampling
    * 3.3.2. Quasi Monte Carlo
    * 3.3.3. 3D Monte Carlo
    * 3.3.4. 2D Gaussian
  * #### 3.4. Advanced Techniques for MCMC
    * 3.4.1. Adaptive Step-Size MCMC
    * 3.4.2. Parallel Chains

### 4. Summary

### 5. Reference

# 1. Introduction and Motivation

* **Background:** $\pi$ is one of the most famous constants in mathematics – defined as the ratio of a circle’s circumference to its diameter. This number is irrational, meaning it cannot be expressed as a simple fraction and its decimal expansion goes on forever without repeating. $\pi$ appears throughout mathematics, science, and engineering: it shows up in geometric formulas (circle area and volume of a sphere), in physics equations (e.g. Einstein’s field equations in general relativity and wave mechanics), and even in everyday technologies like GPS and electronics​. Because $\pi$ is a transcendental number with infinitely many non-repeating decimals, humanity has long been fascinated with determining its value as accurately as possible. From ancient civilizations’ rough estimates to Archimedes’ polygon approximation method and modern computer algorithms, the quest to accurately estimate $\pi$ has been a driving challenge and a benchmark for computational techniques​​.

* **Motivation:** Estimating $\pi$ is an interesting and useful problem because it connects multiple areas of computational mathematics. There are many different ways to arrive at $\pi$, each illuminating a different concept. **Infinite series** provide one approach: for example, the Leibniz series (also known as the Gregory-Leibniz series) expresses $\pi$ as an infinite alternating sum of fractions. **Numerical integration** offers another route by evaluating integrals that equal $\pi$ – for instance, using calculus to compute the area under a curve or the area of a quarter-circle. **Statistical methods** demonstrate a completely different perspective: techniques like Monte Carlo simulation use randomness to approximate $\pi$. In a Monte Carlo approach, one can “throw darts” at a square and see what fraction land inside an inscribed quarter-circle to estimate $\pi$. This blend of methods – analytical series, deterministic calculus, and random simulation – makes $\pi$ estimation a rich educational example that ties together theory and practice across disciplines.

# 2. Four Methods for Estimating $\pi$

## 2.1. Leibniz Series

The Leibniz formula for $\pi$ is an infinite series expression for $\pi$. It states that:

$$
\frac{\pi}{4} = 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} + \cdots = \sum_{k=0}^\infty \frac{(-1)^k}{2k+1}
$$

The terms in this series decrease relatively slowly, which makes convergence to $\pi$ quite gradual. Achieving high precision requires summing a large number of terms. The alternating nature $(-1)^n$ ensures that partial sums oscillate around the true value of $\pi$. By the Alternating Series Test, the error after $N$ terms is bounded by the magnitude of the $(N+1)$-th term.


In [168]:
if __name__ == "__main__":
    iterations = 10_000_000
    start = time.time()
    pi_approx = leibniz_pi(iterations)
    end = time.time()

    print(f"Leibniz Series Approximation of π: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Leibniz Series Approximation of π: 3.1415925535897915
Absolute Error: 1.0000000161269895e-07
Time Taken: 6.288283586502075 seconds


## 2.2. Integral-Based Estimation

Numerical integration provides a deterministic way to approximate definite integrals whose exact values may correspond to $\pi$. One classic example is:

$$
\frac{\pi}{4} = \int_0^1\sqrt{1-x^2}\,dx
$$

which effectively computes the area of a quarter-circle of radius 1. To approximate the integral, we can use **trapezoidal rule**. Each subinterval is treated as a trapezoid rather than a rectangle. If $f(x)$ is the function to integrate, then the sum of trapezoidal areas can be more accurate for smooth functions.

$$
\int_a^b f(x) \, dx \approx \frac{h}{2}\left[f(x_0) + 2\sum_{k=1}^{N-1}f(x_k) + f(x_N)\right]
$$

where $h = \frac{b-a}{2}$.

In [169]:
if __name__ == "__main__":
    N = 10_000_000
    start = time.time()
    pi_approx = trapezoidal_integration(0, 1, N)
    end = time.time()

    print(f"Trapezoidal Rule Approximation of π: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Trapezoidal Rule Approximation of π: 3.1415926535527117
Absolute Error: 3.708144902248023e-11
Time Taken: 7.322698354721069 seconds


## 2.3. Classic Monte Carlo

The **Monte Carlo Rejection Sampling** method uses random uniform sampling to estimate areas by comparing a target area to a known reference area. In this case, we draw points uniformly in the unit square $[0,1]\times[0,1]$ and check whether each point falls inside the quarter of a unit circle drawn within the square. The quarter circle of radius 1 has area $\pi r^2 / 4 = \pi/4$ (since $r=1$), which is exactly $\pi/4$ of the unit square’s area. Because points are uniformly distributed, the probability that a random point lies inside the quarter circle is equal to the area of the quarter circle divided by the area of the square (i.e. $\pi/4$)​. The underlying reason for this method converging $\pi$ is the **Law of Large Numbers**. Each random point can be thought of as a Bernoulli trial that “succeeds” if it lands inside the quarter circle (with success probability $p = \pi/4$) and “fails” otherwise. When we sample many points, the proportion of successes $N_{\text{inside}}/N$ will converge to the true success probability $p$ in the long run​.

In [170]:
if __name__ == "__main__":
    samples = 10_000_000
    start = time.time()
    pi_approx = monte_carlo_pi(samples)
    end = time.time()

    print(f"Monte Carlo Approximation of π: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Monte Carlo Approximation of π: 3.1424308
Absolute Error: 0.0008381464102069636
Time Taken: 4.666754961013794 seconds


## 2.4. Markov Chain Monte Carlo (MCMC)

In traditional Monte Carlo approaches, we sample points $(x,y)$ independently and uniformly in the unit square $[0,1] \times [0,1]$. By counting how many fall in the quarter-circle $x^2 + y^2 \le 1$, we estimate the ratio of areas and thus approximate $\pi$ (because the quarter-circle has area $\pi/4$). However, **MCMC** replaces the idea of “sampling points independently” with a stochastic process that moves from one point $(x,y)$ to another in a dependent manner, forming a Markov chain.

In [171]:
if __name__ == "__main__":
    samples = 10_000_000
    start = time.time()
    pi_approx = mcmc_pi(samples)
    end = time.time()

    print(f"MCMC Approximation of π: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

MCMC Approximation of π: 3.139588
Absolute Error: 0.0020046535897932927
Time Taken: 8.487161874771118 seconds


# 3. Advanced Techniques and Performance Analysis

## 3.1. Advanced Techniques for Leibniz Series

### 3.1.1. Kahan Summation

As previously discussed in 2.1., the Leibniz series converges slowly. Even setting aside the slow convergence, numerical issues also arise from floating-point arithmetic when summing large sequences of positive and negative terms. Round-off errors can accumulate, especially when subtracting nearly equal numbers or adding very small terms to a large running total. A well-known approach, which we also covered in the class, to mitigate round-off error in long summations is **Kahan Summation**. Kahan Summation tracks a “compensation” term to reduce the accumulation of floating-point errors. It is especially helpful in scenarios where a summation has many small terms that can be partially lost when added to a large running total.

In standard floating-point addition, if we add a small number $\delta$ to a much larger number $S$, $\delta$ might be too small to affect the sum in finite precision. Repeatedly adding small terms leads to growing inaccuracies. Kahan Summation addresses this by maintaining a “compensation” variable $c$ that tracks lost low-order bits in each addition. Each time we add a new term:

1. We first subtract the compensation from the current term to produce an adjusted increment $y$.
2. We then add $y$ to the running total $\pi_{\text{est}}$ but carefully reconstruct the error (the difference between the computed sum and the exact sum), storing it back in $c$.

Conceptually, Kahan Summation tries to correct for the small fraction of precision that was “lost” in the previous step, ensuring it is reintroduced in subsequent additions.

In [172]:
if __name__ == "__main__":
    iterations = 10_000_000
    start = time.time()
    pi_approx = leibniz_pi_kahan(iterations)
    end = time.time()

    print(f"Leibniz Series Approximation with Kahan summation of π: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Leibniz Series Approximation with Kahan summation of π: 3.1415925535897933
Absolute Error: 9.999999983634211e-08
Time Taken: 1.9138226509094238 seconds


### 3.1.2. Aitken Formula

**Definition (Aitken Formula):** Given a sequence $\{s_n\}_{n\in\mathbb{N}}$, define

$$
\Delta s_n = s_{n+1} - s_n, \, \Delta^2 s_n = \Delta s_{n+1} \Delta s_n = s_{n+2} - 2s_{n+1} + s_n
$$

Aitken's accelerated sequence $\{s_n^\star\}$ is

$$
s_n^\star = s_n - \frac{(\Delta s_n)^2}{\Delta^2 s_n} \, (\text{assuming $\Delta^2 s_n \ne 0$})
$$

The claim is that if $s_n$ converges to some limit $S$, then $s_n^\star$ often converges more rapidly to $S$.

---

This formula assumes that the sequence’s errors behave in a roughly geometric (linear convergence) manner and solves for the sequence’s apparent limit. In fact, if $s_n$ converges linearly with error $s_n - S \approx C\rho^n$ for large $n$ (with some constant $C$ and ratio $\rho$), then Aitken’s process will exactly eliminate the leading $C\rho^n$ term and give a higher-order error term, thereby dramatically accelerating convergence​.

Now, suppose a sequence $\{s_n\}_{n\in\mathbb{N}}$ can be written as $$s_n = S + A\lambda^n + (\text{smaller terms})$$ where $\lambda$ is some constant. The term $A\lambda^n$ is the leading contribution to the error $s_n - S$. We can check how $\Delta^2s_n$ relates to that error term:

  1. $$\Delta s_n = s_{n+1} - s_n = (S + A\lambda^{n+1}) - (S + A\lambda^n) = A\lambda^n(1-\lambda)$$

  2. $$\Delta^2 s_n = \Delta s_{n+1} - \Delta s_n = A\lambda^{n+1}(1-\lambda) - A\lambda^n(1-\lambda) = A\lambda^n(1-\lambda)^2$$

Therefore, $$\frac{(\Delta s_n)^2}{(\Delta^2 s_n)} = \frac{[A\lambda^n(1-\lambda)]^2}{A\lambda^n(1-\lambda)^2} = A\lambda^n$$ Hence, $$s_n^\star = s_n - \frac{(\Delta s_n)^2}{\Delta^2 s_n} = (S + A\lambda^n) - A\lambda^n = S + (\text{smaller terms})$$ The “main” error term $A\lambda^n$ vanishes in the difference. As a result, $s_n^\star$ is much closer to $S$ than $s_n$ is, especially when $\lambda^n$ dominates the tail.

In [173]:
if __name__ == "__main__":
    iterations = 10_000_000
    start = time.time()
    pi_approx = aitken(iterations)
    end = time.time()

    print(f"Leibniz Series Approximation with Aitken of π: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Leibniz Series Approximation with Aitken of π: 3.1415926535897913
Absolute Error: 1.7763568394002505e-15
Time Taken: 11.251074314117432 seconds


## 3.2. Advanced Techniques for Integral-Based Estimation

### 3.2.1. Simpson's Rule

**Simpson’s 1/3 Rule:** Given an interval $[a,b]$, Simpson’s rule approximates the integral by using three sample points: the two endpoints and the midpoint. Let $m = \frac{a+b}{2}$. We approximate $f(x)$ on $[a,b]$ by a unique quadratic polynomial $P(x)$ that passes through $(a,f(a))$, $(m,f(m))$, and $(b,f(b))$. Integrating this polynomial exactly yields Simpson’s formula:

$$
\int_a^b f(x) \,dx \approx \frac{b-a}{6} \left[f(a) + 4f\left(\frac{a+b}{2} \right) + f(b)\right]
$$

This is known as Simpson’s 1/3 rule. The weights $1$, $4$, and $1$ come from the coefficients of the interpolating parabola. Intuitively, the midpoint $f(m)$ is given quadruple weight because the curved path places more “area” around the center of the interval than a straight-line would. This formula can be derived formally using Lagrange polynomial interpolation or by combining simpler rules: in fact, Simpson’s rule can be obtained as a weighted average of the trapezoidal rule and midpoint rule that cancels out lower-order error terms​. (Specifically, one can show $S_{2n} = \frac{2}{3}M_n + \frac{1}{3}T_n$, meaning the Simpson result on $2n$ subintervals equals a mix of the midpoint ($M$) and trapezoidal ($T$) results​.) The result is a third-order accurate local approximation that integrates cubics exactly.


**Composite Simpson’s Rule:** To approximate $\int_a^b f(x) \,dx$ over a larger interval, we apply the above formula on multiple subintervals. Suppose we divide $[a,b]$ into $N$ equal subintervals of width $h = (b-a)/N$. Important: Simpson’s rule requires that $N$ be even, since each quadratic segment spans two subintervals. We then group the subintervals into pairs and apply Simpson’s 1/3 rule on each pair. If $x_0 = a, x_1 = a+h, x_2 = a+2h, \dots, x_N = b$ are the partition points, the composite Simpson’s rule is:

$$
\int_a^b f(x) \,dx \approx \frac{h}{3} \left[f(x_0) + 4f(x_1) + 2f(x_2) + 4f(x_3) + 2f(x_4) + \cdots + 4f(x_{N-1}) + 2f(x_N)\right]
$$

Here the interior points’ values alternate coefficients $4,2,4,2,\dots,4$ before ending with $f(x_N)$​. This pattern effectively applies the $1,4,1$ Simpson’s weighting on each pair of subintervals. The composite Simpson formula above assumes $N$ is even​ so that the pattern ends correctly with a $4$ coefficient on the second-to-last point and the last point $f(x_N)$ has coefficient $1$. (For example, if $N=6$, the coefficients would be $1,4,2,4,2,4,1$.) The formula shows that points with odd index (the midpoints of each pair) get weight 4, while the even-index interior points get weight 2 in the summation. This method uses $N+1$ function evaluations in total.



---

One of the strength for Composite Simpson's Rule is that if $f$ is sufficiently smooth and has a continuous fourth derivative on $[a,b]$, then the error bound is $$E_\text{simp} = -\frac{(b-a)^5}{180N^4}f^{(4)}(\xi)$$ for some $\xi \in [a,b]$. That indicates an error that goes as $O(N^{-4})$. Hence for the same $N$. Simpson’s rule typically gives a much smaller error than the trapezoidal rule (assuming the integrand is nice and smooth). Conversely, to reach a fixed error tolerance, Simpson’s rule can get away with far fewer intervals $N$.



In [174]:
if __name__ == "__main__":
    N = 10_000_000  # Large number for high accuracy
    start = time.time()
    quarter_circle_area = simpsons_rule_integration(0, 1, N)
    pi_approx = 4 * quarter_circle_area
    end = time.time()

    print(f"Simpson's Rule Approx of pi: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Simpson's Rule Approx of pi: 3.141592653575458
Absolute Error: 1.4335199693960021e-11
Time Taken: 3.4363911151885986 seconds


### 3.2.2. Monte Carlo Integration

**Monte Carlo Integration** is a probabilistic method for approximating definite integrals using random sampling. In contrast to traditional deterministic techniques, Monte Carlo methods rely on randomness to estimate an integral’s value, which is especially useful when an integral is analytically intractable or high-dimensional​. Theoretically speaking, Monte Carlo does not converge as quickly as Simpson’s Rule. Monte Carlo has an error on the order of $1/\sqrt{N}$​, whereas Simpson’s can achieve $1/N^4$ under the right smoothness conditions. That’s a huge difference in convergence rate—Simpson’s is faster to reduce error as $N$ grows. Yet Monte Carlo can often finish sooner (i.e., have “better running time”) in practice. The primary reasons are:

1. Trivial Prallelization:
  * Each Monte Carlo sample is just $(x,y)$ generated at random, so all samples are independent.
  * We can thus throw as many CPU cores or GPUs as we want at the problem.
  * This parallelization can be done without worrying about data dependencies or complicated load balancing.

2. Each Sample is Very Cheap:
  * Checking $x^2 + y^2 \le 1$ is extremely fast—just two multiplications and one comparison.
  * Even at the scale of tens or hundreds of millions of samples, these operations can be done quickly on modern hardware.

3. Dimension Independence:
  * When you do grid‐based methods (like Simpson’s) in higher dimensions, the number of grid points grows as $N^d$, where $d$ is the dimension $N^d$, where $d$ is the dimension. This often explodes extremely fast as $d$ increases.
  * Monte Carlo still maintains an $N^{-1/2}$ convergence even in 10D, 100D, or higher, so you can often handle large‐dimensional problems more feasibly.

In [175]:
if __name__ == "__main__":
    N = 10_000_000  # Number of random samples
    start = time.time()
    pi_approx = monte_carlo_integration(N)
    end = time.time()

    print(f"Monte Carlo Approx of pi: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start} seconds")

Monte Carlo Approx of pi: 3.1414492
Absolute Error: 0.00014345358979328537
Time Taken: 1.97788405418396 seconds


## 3.3. Advanced Techniques for Classic Monte Carlo

### 3.3.1. Stratified Sampling

**Stratified Sampling** introduces a structured approach: the integration domain is partitioned into several disjoint strata, and samples are taken within each stratum rather than from the domain as a whole​. In its simplest form, if the domain is divided into $H$ strata, one allocates a certain number of samples $n_h$ to each stratum $h$ (such that $\sum_{h=1}^H n_h = N$). Within each stratum, points are sampled (usually uniformly if nothing is known about $f$). These per-stratum estimates are then combined (weighted by the size or probability of each stratum) to produce the overall integral estimate. The key difference from standard Monte Carlo is that each region of the domain is guaranteed to be sampled in stratified sampling, rather than leaving the coverage purely to chance. This method is still unbiased (the expected value is still $I$)​, but by reducing the randomness in how samples are geographically distributed, it can achieve a lower estimator variance than crude Monte Carlo. Stratified sampling is often considered a variance-reduction technique (and sometimes categorized as a quasi-Monte Carlo method, since it introduces deterministic structure into the sampling)​.



---

#### Mathematical Foundation

1. **Estimator Formulation:** If the domain $\mathcal{X}$ is partitioned into strata $\mathcal{X}_1, \mathcal{X}_2, \dots, \mathcal{X}_H$, the stratified Monte Carlo estimator can be written as a weighted sum of the averages from each stratum. For example, for a uniform sampling over $\mathcal{X}$, one can write:
$$
\hat{I}_{\text{strat}} = \sum_{h=1}^H w_h \overline{f}_h
$$
where $w_h = \frac{|\mathcal{X}_h|}{|\mathcal{X}|}$ is the weight of stratum $h$, and $\overline{f}_h = \frac{1}{n_h}\sum{i=1}^{n_h} f(x_h,i)$ is the average of $f$ over the $n_h$ samples taken in stratum $h$. If samples in each stratum are drawn uniformly from that stratum, then $\mathbb{E}[\overline{f}_h]$ equals the true average of $f$ over $\mathcal{X}_h$. It follows that $\mathbb{E}[\hat{I}_{\text{strat}}] = \sum_h w_h,\mathbb{E}[\overline{f}_h] = \sum_h w_h I_h = I$, where $I_h = \int{\mathcal{X}_h} f(x)dx$ is the contribution of stratum $h$ to the total integral​. Thus, $\hat{I}_{\text{strat}}$ is an **unbiased** estimator of $I$, just like the standard Monte Carlo estimator.

2. **Variance Decomposition:** The real advantage of stratification lies in its effect on the estimator’s variance. Under independent sampling in each stratum, one can show that the variance of the stratified estimator is the sum of the variances of each stratum’s contribution. Mathematically:
$$
\text{Var}[\hat{I}_\text{strat}] = \sum_{h=1}^H \text{Var}\left(w_h \overline{f}_h\right) = \sum_{h=1}^H w_h^2 \frac{\sigma_h^2}{n_h}
$$
where $\sigma_h^2$ is the variance of $f(X)$ restricted to stratum $h$​. Compare this to the variance of the crude Monte Carlo estimator: $\mathrm{Var}[\hat{I}_\text{MC}] = \frac{\sigma^2}{N}$, where $\sigma^2 = \mathrm{Var}(f(X))$ over the whole domain​. We can relate $\sigma^2$ to the stratification by the **Law of Total Variance**: $\sigma^2 = \sum_{h} w_h,\sigma_h^2 + \sum_h w_h(\mu_h - I)^2$, where $\mu_h$ is the true mean of $f$ in stratum $h$​. The second term $\sum_h w_h(\mu_h - I)^2$ represents the variance between strata (due to different strata having different means).　In a well-executed stratified sampling (especially if we allocate samples proportional to strata sizes), the between-stratum variance is effectively removed from the estimator’s variance. The stratified estimator’s variance becomes $\frac{1}{N}\sum_h w_h,\sigma_h^2$, omitting the between-stratum term​. This is always less than or equal to the crude variance $\frac{\sigma^2}{N}$, since $\sigma^2$ included the extra nonnegative term for between-stratum differences​. In the best case where strata are chosen such that $f$ is nearly constant within each stratum (so that each $\sigma_h^2$ is very low), the variance reduction is significant. In the worst case where the function’s average is the same across all strata (so that dividing was pointless), one finds $\mu_h \approx I$ for all $h$ and the between-stratum term is zero – in that degenerate case stratification offers no variance improvement​. Importantly, under a reasonable allocation of samples to strata, stratified sampling never increases the variance compared to standard Monte Carlo; it will reduce variance unless the stratification did nothing (i.e. all strata had identical means)​.


In [176]:
if __name__ == "__main__":
    strata = 1000
    samples_per_stratum = 10  # total = 1000*1000*10 = 10 million
    start = time.time()
    pi_approx = stratified_sampling_pi(strata, samples_per_stratum)
    end = time.time()

    print(f"Stratified Approx: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time: {end - start} seconds")

Stratified Approx: 3.1415844
Absolute Error: 8.253589792950322e-06
Time: 3.7566256523132324 seconds


### 3.3.2. Quasi Monte Carlo

**Quasi–Monte Carlo** uses deterministic low-discrepancy sequences for choosing sample points instead of random draws. The term “quasi” highlights that QMC points are not truly random, but they approximate the uniform coverage one would expect from random points in the limit. QMC replaces randomness with carefully chosen low-discrepancy sequences that spread out points more uniformly over the integration domain. If $x_1,\dots,x_N$ are selected from a low-discrepancy sequence, we still approximate $I \approx \frac{1}{N}\sum_{i=1}^N f(x_i)$, but we expect a smaller error than a random Monte Carlo sample of the same size. A classic theoretical result called the Koksma–Hlawka inequality formalizes this: for a function $f$ with bounded variation $V(f)$ (in the sense of Hardy–Krause) and points ${x_i}$, the QMC error is bounded by
$$
\left|\int_{\Omega} f(x) \,dx - \frac{1}{N}\sum_{i=1}^N f(x_i)\right| \le V(f) \times D_N^\star
$$
where $D_N^\star$ is the star-discrepancy of the point set. The star-discrepancy $D_N^\star$ measures how far the empirical distribution of points ${x_i}$ deviates from the uniform distribution over $\Omega$. Intuitively, low discrepancy means the points “fill” the domain evenly: any subregion of the domain contains roughly the proportional number of points that it should contain​. Monte Carlo points achieve low discrepancy only on average (with high probability as $N$ grows), whereas QMC sequences are designed to have provably low discrepancy for every $N$.

The main reason QMC often outperforms standard Monte Carlo is the reduction of discrepancy (i.e. improved uniformity) which leads to lower integration error. Monte Carlo’s $1/\sqrt{N}$ error arises from the randomness – points can bunch together or leave large regions unsampled, causing statistical fluctuations. Quasi-random points avoid this. They “cover the domain of interest quickly and evenly”, which means every region gets its fair share of sample points (proportional to its volume) more strictly than with random sampling. This even coverage reduces variance because it prevents the scenario where, by chance, many points fall in an area where $f(x)$ is unusually high or low. In effect, QMC behaves like an extreme form of stratified sampling: the sample points are so uniformly dispersed that they act as if the domain were partitioned into many tiny subregions (strata) with one point in each. Another perspective comes from the law of large numbers and equidistribution: any set of truly random points will eventually cover the domain uniformly on average, but with random fluctuations. Low-discrepancy sequences aim to enforce uniform coverage at every finite $N$, thereby minimizing the worst-case error in approximation. The Koksma–Hlawka inequality mentioned earlier quantifies this: the integration error is at most the product of the function’s variation and the sequence’s discrepancy. For a smooth function (finite variation), using points with discrepancy $D_N$ can drastically shrink the error bound. For example, if $D_N$ is on the order of $(\log N)^s/N$, the bound suggests an error roughly $O((\log N)^s/N)$, which for large $N$ is much smaller than the $O(1/\sqrt{N})$ of Monte Carlo. Even though this worst-case bound can be loose in practice, it reflects the potential of QMC to achieve nearly $O(1/N)$ convergence (up to logarithmic factors) for well-behaved integrands in fixed dimension.


In [177]:
if __name__ == "__main__":
    N = 10_000_000
    start = time.time()
    pi_approx = quasi_monte_carlo_pi(N)
    end = time.time()

    print(f"QMC (Sobol) Approx of pi: {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time: {end - start} seconds")

QMC (Sobol) Approx of pi: 3.1415952
Absolute Error: 2.5464102066941052e-06
Time: 134.91256976127625 seconds


### 3.3.3. 3D Monte Carlo

Consider a unit sphere of radius 1 centered at the origin in 3-dimensional space. This sphere is inscribed in a cube (also centered at the origin) with side length 2 (extending from -1 to 1 along each axis). The volume of the cube is $V_\text{cube} = 8$. The volume of the unit sphere (of radius $r=1$) is known to be:
$$
V_{\text{sphere}} = \frac{4}{3}\pi r^3 = \frac{4}{3}\pi
$$
Now, if one samples a point uniformly at random in the cube $[-1,1]\times[-1,1]\times[-1,1]$, the probability of it landing inside the sphere is equal to the volume fraction of the sphere relative to the cube. In other words,
$$
\Pr(\text{point falls inside unit sphere}) = \frac{V_{\text{sphere}}}{V_{\text{cube}}} = \frac{4\pi/3}{8} = \frac{\pi}{6}
$$
In a Monte Carlo experiment, we draw $N$ random sample points uniformly in the cube and count how many of them fall inside the sphere. Let $N_{\text{inside}}$ be the number of points (out of $N$) satisfying the sphere’s equation $x^2 + y^2 + z^2 \le 1$. By the **Law of Large Numbers**, the ratio $N_{\text{inside}}/N$ will converge to the true probability $\pi/6$ as $N$ becomes large. Thus, an estimator for the sphere’s volume can be obtained from the sample as:
$$
\hat{V}_{\text{sphere}} = \frac{N_{\text{inside}}}{N} \times V_{\text{cube}} = \frac{N_{\text{inside}}}{N} \times 8
$$
This Monte Carlo estimate $\hat V_{\text{sphere}}$ is an approximation of the true sphere volume $4\pi/3$. The larger the number of sample points $N$, the more accurate (on average) $\hat V_{\text{sphere}}$ becomes, with the estimation error decreasing on the order of $1/\sqrt{N}$ (characteristic of Monte Carlo methods).



---

#### Mathematically Interesting Fact about Rejection Rate

The trend observed from 2D to 3D (that the fraction of points inside the sphere decreases) continues as the number of dimensions increases. Geometrically, as the dimension $d$ grows, the volume of the inscribed $d$-dimensional ball (hypersphere) grows more slowly compared to the volume of the surrounding hypercube. In fact, for high $d$, most of the hypercube’s volume resides in the “corners” far from the center, while the hypersphere’s volume is concentrated near the center. The number of rejections — i.e. the proportion of sample points that fall outside the hypersphere — therefore increases with dimension. For a unit $d$-sphere (radius 1) inside a $[-1,1]^d$ hypercube, the volume of the hypercube is $2^d$, while the volume of the $d$-sphere is given by the formula:
$$
V_d(\text{unit sphere}) = \frac{\pi^{d/2}}{\Gamma\!\left(\frac{d}{2} + 1\right)}
$$
where $\Gamma$ is the gamma function (which generalizes factorials). For $d=2$, this gives $V_2 = \pi$; for $d=3$, $V_3 = 4\pi/3$, as expected. The fraction of the hypercube’s volume occupied by the sphere is $\frac{V_d}{2^d}$. Plugging in small dimensions:
* In 2D: $\frac{V_2}{2^2} = \frac{\pi}{4} \approx 0.785$. (About 78.5% of the square is inside the circle.)
* In 3D: $\frac{V_3}{2^3} = \frac{4\pi/3}{8} = \frac{\pi}{6} \approx 0.524$. (About 52.4% of the cube is inside the sphere.)

We see that the volume fraction (and thus acceptance probability) drops rapidly as $d$ increases. By 10 dimensions, the unit sphere occupies only a vanishingly small fraction of the 10-cube (on the order of $0.1%$). For example, in a 10-dimensional Monte Carlo experiment, only about 0.25% of random points lie inside the unit 10-sphere, meaning 99.75% of samples are rejected. This dramatic decline in efficiency is a manifestation of the curse of dimensionality, wherein high-dimensional geometries become challenging for brute-force methods​. In practical terms, as dimension grows, one needs exponentially more samples to get even a few points inside the hypersphere. The Monte Carlo standard error still decreases as $1/\sqrt{N}$, but if only a tiny fraction of $N$ points contribute (fall inside), the absolute error in volume estimation can remain large unless $N$ is extremely large. Thus, the 3D case (with about half the samples useful) is far more efficient than, say, a 10D case (with almost all samples wasted), but still less efficient than the 2D case (where a large majority of samples hit the target region).

In [178]:
if __name__ == "__main__":
    N = 10_000_000  # Number of points to sample
    start = time.time()
    pi_approx = sample_mc_3d(N)
    end = time.time()
    print(f"3D MC Approx of pi (N={N}): {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start:.2f} seconds")

3D MC Approx of pi (N=10000000): 3.1407887999999997
Absolute Error: 0.0008038535897934018
Time Taken: 8.11 seconds


### 3.3.4. 2D Gaussian

We use a 2D Gaussian distribution centered at the origin $(0,0)$ as our proposal distribution $q(x,y)$. This distribution assigns higher probability density to points near the center and lower density to points farther away, which aligns well with the region of interest (the unit circle around the origin). We define:
$$
q(x,y) \;=\; \frac{1}{2\pi\,\sigma^2}\,\exp\Bigl(-\,\frac{x^2 + y^2}{2\,\sigma^2}\Bigr)
$$
where $\sigma^2$ is the variance in each coordinate (the covariance matrix is $\sigma^2 I$). This isotropic Gaussian is a valid probability density over $\mathbb{R}^2$ (its total integral is 1). The choice of $\sigma$ controls how spread out the samples are:
* If $\sigma$ is small, $q$ is tightly concentrated around $(0,0)$, so most samples will fall inside the unit circle (since the circle of radius 1 covers mostly the high-density region of $q$). However, points near the boundary of the circle will have very small $q(x,y)$ values and hence very large weights $1/q(x,y)$.
* If $\sigma$ is large, $q$ is more diffuse; it will sample far outside the circle as well. Many points will fall outside $A$ (contributing zero), and even points inside $A$ might have only moderate weights (since $q$ is not too peaked). This can lead to more wasted samples (those outside) and higher variance.
By choosing a moderate $\sigma$, we ensure $q$ places substantial probability mass inside the unit circle while still not decaying too sharply within $A$. In other words, the Gaussian proposal serves to “focus” sampling around the region $A$ without neglecting any part of it. As a result, more samples contribute meaningfully to the area estimate compared to uniform sampling. Note: It is critical that $q(x,y) > 0$ everywhere on the unit circle; the Gaussian meets this requirement since it has full support on $\mathbb{R}^2$.

To compute the area of a region $A \subset \mathbb{R}^2$ using any PDF $q(x,y)$, observe that

$$
\text{Area}(A)
\;=\;
\iint_{A} \!1\,d(x,y)
\;=\;
\iint_{\mathbb{R}^2}
1_{A}(x,y)\,\frac{1}{q(x,y)}\,q(x,y)\,d(x,y).
$$

Thus, the area can be viewed as an expectation under the distribution $q$:

$$
\hat{\pi}
\;=\;
\text{Area}(A)
\;=\;
\mathbb{E}_{(X,Y)\,\sim\,q}
\Bigl[\,1_{A}(X,Y)\,\frac{1}{q(X,Y)}\Bigr]
\;\approx\;
\frac{1}{N}\,\sum_{i=1}^{N}
\Bigl[\,1_{A}(X_i,Y_i)\,\frac{1}{q(X_i,Y_i)}\Bigr]
$$

where $(X_i,Y_i)$ are i.i.d. samples from $q$. That is, we only add the weight $1/q(X_i,Y_i)$ if the sampled point lies inside the circle, otherwise we add zero. This method is a straightforward application of **importance sampling**, ensuring that sampling from a non-uniform $q$ can still yield an unbiased estimate for the **area** of $A$.


In [179]:
if __name__ == "__main__":
    N = 10_000_00  # 1e7 for a better approximation; try smaller first for speed
    sigma = 0.5
    start = time.time()
    pi_approx = estimate_pi_importance_gaussian(N, sigma)
    end = time.time()

    print(f"Gaussian IS Approx of pi (N={N}, sigma={sigma}): {pi_approx}")
    print(f"Absolute Error: {abs(pi_approx - math.pi)}")
    print(f"Time Taken: {end - start:.2f} seconds")

Gaussian IS Approx of pi (N=1000000, sigma=0.5): 3.1398878093304994
Absolute Error: 0.0017048442592937363
Time Taken: 0.87 seconds


## 3.4. Advanced Techniques for MCMC

### 3.4.1. Adaptive Step-Size MCMC

In a Markov Chain Monte Carlo (MCMC) approach, we generate a sequence of sample points that are dependent on each other, rather than picking each point independently. The key is to design a Markov chain (a random walk in the square) whose stationary distribution is uniform on the unit square​. This ensures that, in the long run, the density of sample points produced by the chain is uniform over the square, just like independent uniform sampling. To construct such a chain, we use the Metropolis algorithm to move through the $(x,y)$ space, deciding whether to accept or reject proposed moves so as to maintain the correct distribution. At each step:

1. **Initialization:**
  * Start $(x,y)$ near the center, or any point inside $[0,1] \times [0,1]$.
  * Choose an initial step size, say $\delta_0 = 0.1$.

2. **Sampling Loop:**
  * $x_{\text{new}} = x_i + U(-\delta_i,\delta_i), y_{\text{new}} = y_i + U(-\delta_i,\delta_i)$.
  * If in $[0,1] \times [0,1]$, accept it: $(x_{i+1},y_{i+1}) = (x_{\text{new}},y_{\text{new}})$. Otherwise, reject: $(x_{i+1},y_{i+1}) = (x_i,y_i)$.
  * Adjust step size $\delta_{i+1}$ based on the acceptance rate so far.

3. **Count if $(x_i,y_i)$ lies in the unit circle**

This procedure is repeated for many iterations, producing a Markov chain $(x_0,y_0)\to(x_1,y_1)\to\cdots$ that explores the unit square. Throughout the simulation we keep track of how often the chain’s points fall inside the quarter circle (i.e. count the number of steps where $x^2+y^2 \le 1$). After a large number of steps, the fraction of samples inside the quarter circle will be our estimate for the probability $\Pr{(x,y)\text{ inside quarter circle}} = \pi$.

In [180]:
if __name__ == "__main__":
    samples = 10_000_000
    start_time = time.time()
    pi_estimate = mcmc_pi_adaptive(samples)
    end_time = time.time()
    print(f"Samples: {samples}, Pi ≈ {pi_estimate}, Error: {abs(pi_estimate - math.pi)}")
    print(f"Time Taken: {end_time - start_time:.3f} seconds")

Samples: 10000000, Pi ≈ 3.1415056, Error: 8.705358979321787e-05
Time Taken: 7.161 seconds


### 3.4.2. Parallel Chains

Each MCMC chain runs independently using a simple Metropolis algorithm to sample points uniformly within the unit square $[0,1] \times [0,1]$. In this setup, the target distribution is uniform over the unit square, so the Metropolis-Hastings rule accepts any proposed move that stays inside the square and rejects moves that would leave the domain​. This ensures each chain individually produces a stream of $(x,y)$ points that are distributed approximately uniformly over the unit square. Crucially, the chains do not interact with each other in any way, so they remain independent in their behavior and output.

To estimate $\pi$, the algorithm leverages the geometry of a quarter circle inscribed in the unit square. In each chain, as points are sampled, we check whether they fall inside the quarter circle of radius 1 (i.e. satisfy $x^2+y^2 \le 1$). The fraction of points that land inside this quarter circle approximates the ratio of the quarter-circle’s area to the area of the whole square​. Since the area of the full unit square is 1 and the area of the quarter circle is $\pi/4$, this fraction should be about $\pi/4$. Thus, an estimate for π is obtained by multiplying the inside-circle fraction by 4​. For example, if a total of $N$ points are sampled in a chain and $M$ of them fell inside the quarter circle, we compute $\pi \approx 4 \times \frac{M}{N}$. As $N$ becomes large, this Monte Carlo estimate converges to the true value of π by the law of large numbers. Because each MCMC chain operates independently and targets the same uniform distribution, we can run multiple chains in parallel without any interference or communication between them. In fact, generating random sample points is an embarrassingly parallel task: the generation of each point (or each chain’s sequence of points) is independent of all others​. This independence means that samples drawn by different chains have no correlation with each other, which avoids the issues of autocorrelation that occur between successive samples within a single chain. With independent chains, we can safely collect results from many samplers running simultaneously, effectively increasing the total number of samples gathered in a given time. In other words, running $k$ chains at once can produce roughly $k$ times more samples in the same wall-clock time, since (conditional on different random seeds) the chains are independent and can be simulated concurrently​.

After all chains have run (or periodically during execution), the results from the separate chains are combined to form a global estimate of π. Since each chain counted how many of its points landed inside the quarter circle, we can sum these counts across all chains and likewise sum the total number of points sampled across all chains. If chain 1 had $M_1$ hits out of $N_1$ samples, chain 2 had $M_2$ out of $N_2$​, and so on, then overall we have $M_{\text{total}} = M_1 + M_2 + \cdots$ inside-circle points out of $N_{\text{total}} = N_1 + N_2 + \cdots$ total points. The global estimate of π is then calculated as $4 \times \frac{M_{\text{total}}}{N_{\text{total}}}$​. This procedure is equivalent to pooling all the samples together: it does not matter whether the $N_{\text{total}}$ points came from one long chain or from several independent chains. In fact, Monte Carlo estimators are additive, so we can “add all of these together before estimating $\pi$" - effectively taking an average across chains. By combining the chains’ outcomes in this way, we obtain a single, more precise estimate. Each chain’s contribution improves the overall accuracy, and the uncertainty of the final estimate decreases as we aggregate more independent samples.

In [181]:
if __name__ == "__main__":
    total_samples = 10_000_000
    num_procs = 4  # number of parallel processes (cores)

    # We split total_samples across processes
    samples_per_proc = total_samples // num_procs

    start = time.time()
    with mp.Pool(processes=num_procs) as pool:
        # Launch parallel tasks
        results = pool.starmap(
            mcmc_chain,
            [(samples_per_proc, 0.1, 42 + i) for i in range(num_procs)]  # seeds to vary
        )

    total_inside = sum(results)
    pi_estimate = 4.0 * total_inside / (samples_per_proc * num_procs)

    end = time.time()

    print(f"Parallel MCMC Approx of pi: {pi_estimate}")
    print(f"Absolute Error: {abs(pi_estimate - math.pi)}")
    print(f"Time Taken: {end - start:.3f} seconds using {num_procs} processes")

Parallel MCMC Approx of pi: 3.13761
Absolute Error: 0.003982653589793106
Time Taken: 6.281 seconds using 4 processes


# 4. Summary

* **Leibniz Series:** While Aitken Acceleration provides the best accuracy by far, it requires more computational effort, leading to the longest execution time. Kahan Summation offers a middle ground—retaining good accuracy (near $10^{-7}$) but with notably reduced computational time compared to the basic Leibniz series implementation.

* **Integration:** Both Trapezoidal and Simpson’s Rule are highly accurate for these tasks, with Simpson’s Rule edging out slightly in precision. The Monte Carlo method is the quickest but comes with a higher error magnitude unless significantly more samples are used.

* **Monte Carlo:** QMC provides the highest accuracy but is computationally expensive, while Gaussian Importance Sampling is the fastest but least accurate with the chosen parameters. Stratified Sampling offers a good balance of relatively low error and moderate running time.

* **MCMC:** Adaptive MCMC strikes the best balance here, achieving the highest accuracy and a shorter runtime than standard MCMC. Parallel MCMC can reduce total wall-clock time, though at the cost of higher error, depending on parameters and how the chains are managed.



---



Overall, mathematically driven methods (Leibniz Series and Integration) achieve very high accuracy ($10^{-11}$ to $10^{-15}$) but may require significant iteration counts, clever summation techniques, or dense subdivisions. Often more “exact” in their convergence properties but can be slower at extreme precisions. Monte Carlo methods and MCMC were faster to implement and easily parallelized, but the variance can be large; high accuracy demands a lot of samples.

# 5. Reference

[1] [Wikipedia: Leibniz Series](https://en.wikipedia.org/wiki/Leibniz_formula_for_%CF%80#:~:text=Taylor%20series%20%20for%20the,1%29%5E%7Bk%7Dx%5E%7B2k%2B1%7D%7D%7B2k%2B1)

[2] [Trapezoidal Rule](https://math.libretexts.org/Courses/Community_College_of_Denver/MAT_2420_Calculus_II/03%3A_Techniques_of_Integration/3.06%3A_Numerical_Integration)

[3] [Wikipedia: Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method)

[4] [Wikipedia: MCMC](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo)

[5] [Wikipedia: Aitken Process](https://en.wikipedia.org/wiki/Aitken%27s_delta-squared_process#:~:text=squared%20process%20associates%20to%20this,sequence%20the%20new%20sequence)

[6] [Aitken Formula](https://francisbach.com/acceleration-without-pain/#:~:text=%5C%28x%20%5Cmapsto%20%5Cfrac%7B1%7D%7B1%2Bx,3.14%7D27%20%5C%5C%208)

[7] [Stratified Sampling](https://xuk.ai/blog/stratified-sampling.html#:~:text=1)

[8] [Low Descrepency Sequence](https://en.wikipedia.org/wiki/Low-discrepancy_sequence#:~:text=Roughly%20speaking%2C%20the%20discrepancy%20of,by%20taking%20the%20worst%20value)

[9] [2D Gaussian](https://builtin.com/articles/importance-sampling#:~:text=Where%20,offset%20the%20probability%20of%20sampling)

[10] [Adaptive MCMC](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=503dd21a40c966736df809a96c8e31277337e59d#:~:text=A%20simple%20way%20to%20avoid,Figure%203)

[11] [Parallel Chains](https://bede-documentation.readthedocs.io/en/latest/guides/wanderings/Estimating-pi-in-CUDALand.html#)